# OpenEO endpoint in Examind - Example

By Quentin BIALOTA (Geomatys)

Contact : quentin.bialota@geomatys.com

---

### **/!\ WARNING**
This initial implementation of openEO is not yet complete and may contain bugs. Please let me know if you find any. 

For the moment we are not compatible with openEO clients. In this tutorial we will use http requests to communicate with the examind openEO implementation.

---

**Prerequisites :**
- Have an examind (via docker) running on your machine
- Have imported sentinel-2 data in Examind (through the web ui http://localhost/8080/examind [login: admin / password: admin])

In [67]:
# Imports (Execute this cell once)
import requests
import json

# Set the server ip here (before the /examind), by default: localhost:8080
SERVER_IP = "http://localhost:8080"

### 1 - List openEO available processes

The first step we can do is to request the list of processes available via examind's openEO endpoint.

In [ ]:
URL = SERVER_IP + "/examind/WS/openeo/process/WPS/processes"

r = requests.get(url = URL)

data = r.json()

# If you want to show the raw json output
# print(json.dumps(data, indent=2))

# Iterate over each process in the data
for process in data['processes']:
    print(f"ID: {process['id']}")

    print(f"Description: {process['description']}")

    print("Parameters:")
    for parameter in process['parameters']:
        print(f"- {parameter['name']} {parameter['schema']['type']}")
        print(f"  Description: {parameter['description']}")
        if 'subtype' in parameter['schema'] and parameter['schema']['subtype'] is not None:
            print(f"  Subtype: {parameter['schema']['subtype']}")

    print("Returns:")
    if process['returns']['schema'] is not None:
        print(f"  Description: {process['returns']['description']}")
        print(f"  Type: {process['returns']['schema']['type']}")
        if 'subtype' in process['returns']['schema'] and process['returns']['schema']['subtype'] is not None:
            print(f"  Subtype: {process['returns']['schema']['subtype']}")

    print("\n")

You can also get a list of processes that have been added by a user.

*(If you follow this tutorial in order, you shouldn't find anything for the moment.
If you come back to this request later, you'll see the process we're going to create next.)*

In [ ]:
URL = SERVER_IP + "/examind/WS/openeo/process/WPS/process_graphs"

r = requests.get(url = URL)

data = r.json()

# If you want to show the raw json output
# print(json.dumps(data, indent=2))

# Iterate over each process in the data
for process in data['processes']:
    print(f"ID: {process['id']}")

    print(f"Description: {process['description']}")

    print("Parameters:")
    for parameter in process['parameters']:
        print(f"- {parameter['name']} {parameter['schema']['type']}")
        print(f"  Description: {parameter['description']}")
        if 'subtype' in parameter['schema'] and parameter['schema']['subtype'] is not None:
            print(f"  Subtype: {parameter['schema']['subtype']}")

    print("Returns:")
    if process['returns']['schema'] is not None:
        print(f"  Description: {process['returns']['description']}")
        print(f"  Type: {process['returns']['schema']['type']}")
        if 'subtype' in process['returns']['schema'] and process['returns']['schema']['subtype'] is not None:
            print(f"  Subtype: {process['returns']['schema']['subtype']}")

    print("\n")

### 2 - Create your own process

This step should eventually work via openEO web Editor (https://github.com/Open-EO/openeo-web-editor), but for the moment our implementation is not compatible (nor is it compatible with openEO clients) because some of the paths to the APIs are not standard (the way the API works, however, is compliant).

For the moment, we're going to create an EVI (Enhanced Vegetation Index) process by ourselves, directly via an http request !

In [38]:
# Set Json of my EVI-Sentinel process

data = {
    'id': 'evi-sentinel',
    'summary': 'Enhanced Vegetation Index',
    'description': 'Computes the Enhanced Vegetation Index (EVI). It is computed with the following formula: `2.5 * (NIR - RED) / (1 + NIR + 6*RED + 7.5*BLUE)`.',
    'parameters': [
        {
            'name': 'dataId',
            'description': 'Sentinel-2 data id',
            'schema': {
                'type': 'String'
            }
        },
        {
            'name': 'bbox',
            'description': 'Spatial extent',
            'schema': {
                'type': 'Object'
            }
        },
        {
            'name': 'temporal',
            'description': 'Temporal extent',
            'schema': {
                'type': 'Array',
                'subtype': 'String'
            }
        },
        {
            'name': 'bands',
            'description': 'Bands to load',
            'schema': {
                'type': 'Array',
                'subtype': 'String'
            }
        }
    ],
    'returns': {
        'description': 'Computed EVI.',
        'schema': {
            'type': 'object'
        }
    },
    'process_graph': {
        'load': {
            'process_id': 'examind.coverage.load',
            'arguments': {
                'coverageId': {
                    'from_parameter': 'dataId'
                },
                'spatial_extent': {
                    'from_parameter': 'bbox'
                },
                'temporal_extent': {
                    'from_parameter': 'temporal'
                },
                'bands': {
                    'from_parameter': 'bands'
                }
            }
        },
        'bs3': {
            'process_id': 'geotoolkit.coverage:bandselect',
            'arguments': {
                'coverage': {
                    'from_node': 'load'
                },
                'bands': [
                    1
                ]
            }
        },
        'bs4': {
            'process_id': 'geotoolkit.coverage:bandselect',
            'arguments': {
                'coverage': {
                    'from_node': 'load'
                },
                'bands': [
                    2
                ]
            }
        },
        'bs8': {
            'process_id': 'geotoolkit.coverage:bandselect',
            'arguments': {
                'coverage': {
                    'from_node': 'load'
                },
                'bands': [
                    3
                ]
            }
        },
        'sub': {
            'process_id': 'examind.coverage:math:substract',
            'arguments': {
                'first': {
                    'from_node': 'bs8'
                },
                'second': {
                    'from_node': 'bs3'
                }
            }
        },
        'p1': {
            'process_id': 'examind.coverage:math:multiplyWithValue',
            'arguments': {
                'value': 6.0,
                'coverage': {
                    'from_node': 'bs3'
                }
            }
        },
        'p2': {
            'process_id': 'examind.coverage:math:multiplyWithValue',
            'arguments': {
                'value': 7.5,
                'coverage': {
                    'from_node': 'bs4'
                }
            }
        },
        'sumCoverages1': {
            'process_id': 'examind.coverage:math:sum',
            'arguments': {
                'first': {
                    'from_node': 'bs8'
                },
                'second': {
                    'from_node': 'p1'
                }
            }
        },
        'sumCoverages2': {
            'process_id': 'examind.coverage:math:sum',
            'arguments': {
                'first': {
                    'from_node': 'sumCoverages1'
                },
                'second': {
                    'from_node': 'p2'
                }
            }
        },
        'sum': {
            'process_id': 'examind.coverage:math:sumWithValue',
            'arguments': {
                'coverage': {
                    'from_node': 'sumCoverages2'
                },
                'value': 1.0
            }
        },
        'div': {
            'process_id': 'examind.coverage:math:divide',
            'arguments': {
                'first': {
                    'from_node': 'sub'
                },
                'second': {
                    'from_node': 'sum'
                }
            }
        },
        'p3': {
            'process_id': 'examind.coverage:math:multiplyWithValue',
            'arguments': {
                'value': 2.5,
                'coverage': {
                    'from_node': 'div'
                }
            }
        },
        'save': {
            'process_id': 'examind.coverage.save_result',
            'arguments': {
                'format': 'GTIFF',
                'coverage': {
                    'from_node': 'p3'
                }
            },
            'result': True
        }
    }
}

You can check if your process is valid with `/validation` endpoint

In [39]:
URL = SERVER_IP + "/examind/WS/openeo/process/WPS/validation"

headers = {'Content-Type': 'application/json', 'Accept': '*/*'}

r = requests.post(url=URL, data=json.dumps(data), headers=headers)

print(f"Status {r.status_code}")
print(json.dumps(r.json(), indent=2))

Status 200
{
  "errors": [
    {
      "id": "360697e8-27a0-453d-951d-5b437cba3af3",
      "code": "ProcessIDAlreadyExist",
      "message": "The process id specified already exist, you cannot use the same to add this process.",
      "links": []
    }
  ]
}


Then, if it's valid, you can put your process on the server with `/process_graphs/{id_process}` endpoint

In [65]:
URL = SERVER_IP + "/examind/WS/openeo/process/WPS/process_graphs/evi-sentinel"

headers = {'Content-Type': 'application/json', 'Accept': '*/*'}

r = requests.put(url=URL, data=json.dumps(data), headers=headers)

print(f"Status {r.status_code}")
if r.text and r.text is not None:
    print(json.dumps(r.json(), indent=2))

Status 200


**NOTE 1**

The process id you created is not exactly the same as the one specified. The server adds the prefix `openeo-` before your process id for internal management reasons. 
So, for example, `evi-sentinel` becomes `openeo-evi-sentinel`.

---

**NOTE 2**

If the process id already exist because you have already uploaded this process, you can delete it with the following request :

In [64]:
URL = SERVER_IP + "/examind/WS/openeo/process/WPS/process_graphs/evi-sentinel"

headers = {'Content-Type': 'application/json', 'Accept': '*/*'}

r = requests.delete(url=URL, headers=headers)

print(f"Status {r.status_code}")

Status 200


### 3 - Execute your process (sync way)

You can run an openEo process in two ways :
- **synchronised** (the result arrives directly, you only have to make one request)
- **asynchronous** (you launch the process, and you have to run another request to see the status and the result).

Here, I will show you how to execute our *evi-sentinel* process with one request (sync).

We will use the `/result` endpoint

---

**NOTE**

In the case of synchronous execution, the user must create a process_graph passed in the request. A previously created process_graph cannot be queried directly. A workaround is to create a process_graph which takes only one process, the one created before (openeo-evi-sentinel).

---

**INFO**

**1. How to import my data in Examind ?**

I've written a short tutorial to import your data in Examind, using the sentinel-2 data example. Here is the link : [import_data.md](./import_data.md)

**2. How do you retrieve the id of the data you want to use ?**

I've written a short tutorial to find the data id, using the sentinel-2 data example. Here is the link : [retrieve_data_id.md](./retrieve_data_id.md)

In [74]:
URL = SERVER_IP + "/examind/WS/openeo/process/WPS/result"

headers = {'Content-Type': 'application/json', 'Accept': '*/*'}

exec_data = {
    "id": "evi-execution",
    "summary": "Enhanced Vegetation Index",
    "description": "Computes the Enhanced Vegetation Index (EVI). It is computed with the following formula: `2.5 * (NIR - RED) / (1 + NIR + 6*RED + -7.5*BLUE)`.",
    "parameters": [],
    "returns": {
        "description": "Computed EVI.",
        "schema": {
            "type": "object"
        }
    },
    "process_graph": {
        "load": {
            "process_id": "examind-dynamic.openeo-evi-sentinel",
            "arguments": {
                "dataId": "sentinel_b03_b04_b08:1",
                "bbox": {
                    "west": 3.87,
                    "east": 4.5,
                    "north": 43.56,
                    "south": 43.56
                },
                "temporal": ["test", "test"],
                "bands": ["Grayscale"]
            },
            "result": "true"
        }
    }
}

r = requests.post(url=URL, data=json.dumps(data), headers=headers)

print(f"Status {r.status_code}")
if r.text and r.text is not None:
    print(r.text)

Status 404
Service ID : WPSnot found
